In [1]:
import pyomo.environ as pyEnv

In [2]:
p_rt = [[260000, 280000, 380000, 350000, 230000, 150000, 0, 0, 0, 0, 140000, 180000], 
     [380000, 370000, 350000, 230000, 120000, 0, 0, 0, 0, 110000, 250000, 360000], 
     [370000, 400000, 420000, 380000, 210000, 110000, 0, 0, 0, 0, 130000, 240000], 
     [260000, 250000, 200000, 120000, 0, 0, 0, 0, 100000, 110000, 150000, 250000]]

In [3]:
ep_f = [240000,180000,160000,170000]

In [4]:
cp_f = [10.3,12.4,9.7,10.5]

In [5]:
eb_f = [1500000,750000,1250000,1000000]

In [6]:
cb_f =[8.10,7.20,7.60,5.40]

In [7]:
fp_f = [1000000,800000,1000000,900000]

In [8]:
cfp_f = [5.3,4.1,4.6,3.8]

In [9]:
ct_rf =[[10.8, 12, 7.2, 26], 
       [3.6, 4, 6.8, 25.2], 
       [10.8, 9.2, 14.4, 21.6], 
       [16.4, 14.8, 23.4, 10.8]]

In [10]:
ct_ffLinha =[[0, 7.2, 8.4, 18.4], 
       [7.2, 0, 10, 17.2], 
       [8.4, 10, 0, 22.8], 
       [18.4, 17.2, 22.8, 0]]

In [11]:
d_t = [600000, 500000, 500000, 400000, 300000, 500000, 700000, 900000, 800000, 800000, 700000, 600000]

In [12]:
#indices
r=[1,2,3,4]
f=[1,2,3,4]
fLinha=[1,2,3,4]
t=[1,2,3,4,5,6,7,8,9,10,11,12]

In [13]:

modelo = pyEnv.ConcreteModel()

modelo.R = pyEnv.RangeSet(len(r))
modelo.F = pyEnv.RangeSet(len(f))
modelo.FLinha = pyEnv.RangeSet(len(fLinha))
modelo.T = pyEnv.RangeSet(len(t))



modelo.x_rft = pyEnv.Var(modelo.R,modelo.F,modelo.T,within = pyEnv.NonNegativeReals)
modelo.eb_ft = pyEnv.Var(modelo.F,modelo.T, within = pyEnv.NonNegativeReals)
modelo.y_ft = pyEnv.Var(modelo.F,modelo.T,within = pyEnv.NonNegativeReals)
modelo.ep_ft = pyEnv.Var(modelo.F,modelo.T,within = pyEnv.NonNegativeReals)
modelo.z_fLinhaft = pyEnv.Var(modelo.FLinha,modelo.F, modelo.T , within = pyEnv.NonNegativeReals)
modelo.h_fLinhaft = pyEnv.Var(modelo.FLinha,modelo.F, modelo.T,within = pyEnv.NonNegativeReals)
modelo.s_ft = pyEnv.Var(modelo.F,modelo.T,within = pyEnv.NonNegativeReals)
modelo.he = pyEnv.Var(modelo.FLinha,modelo.T,within = pyEnv.NonNegativeReals)
modelo.ze = pyEnv.Var(modelo.F,modelo.T,within = pyEnv.NonNegativeReals)

modelo.P_rt = pyEnv.Param(modelo.R,modelo.T,initialize = lambda modelo,r,t:p_rt[r-1][t-1])
modelo.EP_f = pyEnv.Param(modelo.F,initialize = lambda modelo,f:ep_f[f-1])
modelo.CP_f = pyEnv.Param(modelo.F,initialize = lambda modelo,f:cp_f[f-1])
modelo.EB_f = pyEnv.Param(modelo.F,initialize = lambda modelo,f:eb_f[f-1])
modelo.CB_f = pyEnv.Param(modelo.F,initialize = lambda modelo,f:cb_f[f-1])
modelo.Fp_f = pyEnv.Param(modelo.F,initialize = lambda modelo,f:fp_f[f-1])
modelo.CFP_f = pyEnv.Param(modelo.F,initialize = lambda modelo,f:cfp_f[f-1])
modelo.CT_rf = pyEnv.Param(modelo.R,modelo.F,initialize = lambda modelo,r,f:ct_rf[r-1][f-1])
modelo.CT_FLinhaF =  pyEnv.Param(modelo.FLinha,modelo.F,initialize = lambda modelo,f,fLinha:ct_ffLinha[f-1][fLinha-1])
modelo.D_t =  pyEnv.Param(modelo.T,initialize = lambda modelo,t:d_t[t-1])

#Função objetivo
def func_objetivo(modelo):
    return sum(modelo.x_rft[r,f,t] * modelo.CT_rf[r,f] for r in modelo.R for f in modelo.F for t in modelo.T) + sum(
        modelo.eb_ft[f,t] * modelo.CB_f[f] for f in modelo.F for t in modelo.T) + sum(
        modelo.h_fLinhaft[fLinha,f,t] * modelo.CT_FLinhaF[fLinha,f] for fLinha in modelo.FLinha for f in modelo.F for t in modelo.T) + sum(
        modelo.y_ft[f,t] * modelo.CFP_f[f] for f in modelo.F for t in modelo.T) + sum(
        modelo.z_fLinhaft[fLinha,f,t] * modelo.CT_FLinhaF[fLinha,f] for fLinha in modelo.FLinha for f in modelo.F  for t in modelo.T) + sum(
        modelo.ep_ft[f,t] * modelo.CP_f[f] for f in modelo.F for t in modelo.T)
    

modelo.objetivo = pyEnv.Objective(rule = func_objetivo ,sense = pyEnv.minimize) 

#Capacidade de distribuição de fumo bruto
def CapacidadeDistribuicaoDeFumoBruto(modelo,r,t):
    return modelo.P_rt[r,t] >= sum(modelo.x_rft[r,f,t] for f in modelo.F) 

modelo.capacidadeDistribuicaoDeFumoBruto = pyEnv.Constraint(modelo.R,modelo.T,rule = CapacidadeDistribuicaoDeFumoBruto)

#Estoque Bruto e disponível para transporte
def EstoqueBrutoDisponivelParaTransporte(modelo,f,t): 
    if t > 1 :
        return  modelo.eb_ft[f,t] == modelo.eb_ft[f,t-1] + sum(
            modelo.x_rft[r,f,t] for r in modelo.R) + sum(
            modelo.h_fLinhaft[fLinha,f,t] for fLinha in modelo.FLinha if f != fLinha) -sum(
         modelo.h_fLinhaft[fLinha,f,t] for fLinha in modelo.FLinha if f != fLinha) - modelo.he[f,t] - modelo.y_ft[f,t]
    else:
         return  modelo.eb_ft[f,t] ==  modelo.eb_ft[f,t] + sum(modelo.x_rft[r,f,t] for r in modelo.R) + sum(
            modelo.h_fLinhaft[fLinha,f,t] for fLinha in modelo.FLinha if f != fLinha) - modelo.he[f,t] - modelo.y_ft[f,t]

modelo.estoqueBrutoDisponivelParaTransporte = pyEnv.Constraint(modelo.F,modelo.T, rule = EstoqueBrutoDisponivelParaTransporte)

def f_restr4(modelo, fLinha, f, t):
    if fLinha == f:
        return modelo.he[f,t] == sum(modelo.h_fLinhaft[fLinha,f,t] for f in modelo.F)
    else:
        return pyEnv.Constraint.Skip
    
modelo.restr_4 = pyEnv.Constraint(modelo.FLinha, modelo.F, modelo.T, rule=f_restr4)

#Estoque processado e disponível para transporte
def EstoqueProcessadoDisponivelTransporte(modelo,f,t):
    if t > 1 :
        return modelo.ep_ft[f,t] == modelo.y_ft[f,t] + modelo.ep_ft[f,t-1] + sum(
            modelo.z_fLinhaft[fLinha,f,t] for fLinha in modelo.FLinha  if f != fLinha ) - sum(
            modelo.z_fLinhaft[f,fLinha,t] for fLinha in modelo.FLinha if f != fLinha)
    else:
        return modelo.ep_ft[f,t] == modelo.ep_ft[f,t] + modelo.y_ft[f,t] + sum(
            modelo.z_fLinhaft[fLinha,f,t] for fLinha in modelo.FLinha if f != fLinha) - sum(
            modelo.z_fLinhaft[f,fLinha,t] for fLinha in modelo.FLinha if f != fLinha)

modelo.estoqueProcessadoDisponivelTransporte = pyEnv.Constraint(modelo.F, modelo.T, rule = EstoqueProcessadoDisponivelTransporte)

#Demanda
def Demanda(modelo,t):
    return modelo.D_t[t] == sum(modelo.s_ft[f,t] for f in modelo.F)

modelo.demanda = pyEnv.Constraint(modelo.T, rule = Demanda)    


#Capacidade Estoque bruto
def CapacidadeEstoqueBruto(modelo,f,t):
    return modelo.eb_ft[f,t] <= modelo.EB_f[f]

modelo.capacidadeEstoqueBruto = pyEnv.Constraint(modelo.F, modelo.T,rule = CapacidadeEstoqueBruto)

#Capacidade de processamento
def CapacidadeProcessamento(modelo,f,t):
    return modelo.y_ft[f,t] <= modelo.Fp_f[f]

modelo.capacidadeProcessamento = pyEnv.Constraint(modelo.F, modelo.T,rule = CapacidadeProcessamento)


#Capacidade do Estoque processamento
def CapacidadeEstoqueProcessado(modelo,f,t):
    return modelo.ep_ft[f,t] <= modelo.EP_f[f]

modelo.capacidadeEstoqueProcessado = pyEnv.Constraint(modelo.F, modelo.T,rule = CapacidadeEstoqueProcessado)



In [14]:
solver = pyEnv.SolverFactory('glpk')
resultado_objetivo = solver.solve(modelo,tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\User\AppData\Local\Temp\tmpfz4ln9h5.glpk.raw --wglp C:\Users\User\AppData\Local\Temp\tmp9r261l7f.glpk.glp
 --cpxlp C:\Users\User\AppData\Local\Temp\tmpzmcrbwjk.pyomo.lp
Reading problem data from 'C:\Users\User\AppData\Local\Temp\tmpzmcrbwjk.pyomo.lp'...
349 rows, 769 columns, 1437 non-zeros
3884 lines were read
Writing problem data to 'C:\Users\User\AppData\Local\Temp\tmp9r261l7f.glpk.glp'...
3387 lines were written
GLPK Simplex Optimizer 5.0
349 rows, 769 columns, 1437 non-zeros
Preprocessing...
188 rows, 628 columns, 1088 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 180
*     0: obj =   0.000000000e+00 inf =   0.000e+00 (98)
*    63: obj =   0.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.6 Mb (652416 byt

In [15]:
print(resultado_objetivo)


Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 349
  Number of variables: 769
  Number of nonzeros: 1437
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06034135818481445
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [16]:
for t in modelo.T:
    print("mes",t-1)
    for r in modelo.R:
        for f in modelo.F:
            print("De", r-1, "para", f-1, "----->", modelo.x_rft[r,f,t]())
        print("")

mes 0
De 0 para 0 -----> 0.0
De 0 para 1 -----> 0.0
De 0 para 2 -----> 0.0
De 0 para 3 -----> 0.0

De 1 para 0 -----> 0.0
De 1 para 1 -----> 0.0
De 1 para 2 -----> 0.0
De 1 para 3 -----> 0.0

De 2 para 0 -----> 0.0
De 2 para 1 -----> 0.0
De 2 para 2 -----> 0.0
De 2 para 3 -----> 0.0

De 3 para 0 -----> 0.0
De 3 para 1 -----> 0.0
De 3 para 2 -----> 0.0
De 3 para 3 -----> 0.0

mes 1
De 0 para 0 -----> 0.0
De 0 para 1 -----> 0.0
De 0 para 2 -----> 0.0
De 0 para 3 -----> 0.0

De 1 para 0 -----> 0.0
De 1 para 1 -----> 0.0
De 1 para 2 -----> 0.0
De 1 para 3 -----> 0.0

De 2 para 0 -----> 0.0
De 2 para 1 -----> 0.0
De 2 para 2 -----> 0.0
De 2 para 3 -----> 0.0

De 3 para 0 -----> 0.0
De 3 para 1 -----> 0.0
De 3 para 2 -----> 0.0
De 3 para 3 -----> 0.0

mes 2
De 0 para 0 -----> 0.0
De 0 para 1 -----> 0.0
De 0 para 2 -----> 0.0
De 0 para 3 -----> 0.0

De 1 para 0 -----> 0.0
De 1 para 1 -----> 0.0
De 1 para 2 -----> 0.0
De 1 para 3 -----> 0.0

De 2 para 0 -----> 0.0
De 2 para 1 -----> 0.0
De 2 p